##### Как работаю основные слои

In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import Embedding, Conv1D, LSTM

# Основные слои

### Input и Dense  

Слой **Input** инициализирует тензор Keras  

Слой **Dense** реализует операцию: output = activation(dot(input, weights) + bias), где:  
activation — это функция активации по элементам,  
weights — это матрица весов слоя,  
bias — это вектор смещения слоя.

In [2]:
inp = Input(shape=(64))
x = Dense(200, activation='relu')(inp)
x = Dense(10, activation='softmax')(x)
 
model = Model(inp, x)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense (Dense)               (None, 200)               13000     
                                                                 
 dense_1 (Dense)             (None, 10)                2010      
                                                                 
Total params: 15,010
Trainable params: 15,010
Non-trainable params: 0
_________________________________________________________________


Слой Dense может работать не только с вектором, но и с матрицей.

In [3]:
inp = Input(shape=(64, 64))
x = Dense(100, activation='relu')(inp)
x = Dense(2)(x)
 
model = Model(inp, x)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64)]          0         
                                                                 
 dense_2 (Dense)             (None, 64, 100)           6500      
                                                                 
 dense_3 (Dense)             (None, 64, 2)             202       
                                                                 
Total params: 6,702
Trainable params: 6,702
Non-trainable params: 0
_________________________________________________________________


In [4]:
# входные данные
input_data = np.array([[[2,4,6,8],
                        [2,4,6,8],
                        [2,4,6,8]],
                       
                       [[3,5,7,9],
                        [3,5,7,9],
                        [3,5,7,9]]])

# Матрица весов: два нейрона с четырьмя весовыми коэффициентами
weights = np.array([[0.3, 0.5],
                    [0.3, 0.5],
                    [0.3, 0.5],
                    [0.3, 0.5]])

print("Размер матрицы выходных данных:", input_data.shape) 
print("Размер матрицы весов:", weights.shape)

Размер матрицы выходных данных: (2, 3, 4)
Размер матрицы весов: (4, 2)


In [5]:
output = input_data.dot(weights)  # аналогично input_data @ weights
output

array([[[ 6. , 10. ],
        [ 6. , 10. ],
        [ 6. , 10. ]],

       [[ 7.2, 12. ],
        [ 7.2, 12. ],
        [ 7.2, 12. ]]])

In [6]:
print("Размер выходных данных:", output.shape)

Размер выходных данных: (2, 3, 2)


### Flatten

Слой **Flatten** вытягивает данные в вектор. Не трогает размерность, отвечающую за батч.

In [7]:
inp = Input(shape=(64,64))
x = Dense(100)(inp)
x = Flatten()(x)
model = Model(inp, x)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64)]          0         
                                                                 
 dense_4 (Dense)             (None, 64, 100)           6500      
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
Total params: 6,500
Trainable params: 6,500
Non-trainable params: 0
_________________________________________________________________


### Reshape

Слой **Reshape** приводит тензор к указанной форме.

In [8]:
inp = Input(shape=(64,64))
x = Dense(100)(inp)
x = Reshape((-1,))(x)
 
model = Model(inp, x)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64)]          0         
                                                                 
 dense_5 (Dense)             (None, 64, 100)           6500      
                                                                 
 reshape (Reshape)           (None, 6400)              0         
                                                                 
Total params: 6,500
Trainable params: 6,500
Non-trainable params: 0
_________________________________________________________________


In [9]:
inp = Input(shape=(64))
x = Dense(100)(inp)
x = Reshape((10,10))(x)
 
model = Model(inp, x)

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_6 (Dense)             (None, 100)               6500      
                                                                 
 reshape_1 (Reshape)         (None, 10, 10)            0         
                                                                 
Total params: 6,500
Trainable params: 6,500
Non-trainable params: 0
_________________________________________________________________


### Concatenate

Слой **Concatenate** объединяет данные с разных ветвей нейронной сети.

In [10]:
inp_1 = Input(shape=(64, 64, 3))
inp_2 = Input(shape=(64, 64, 3))
x = Concatenate(axis=3)([inp_1, inp_2])
 
model = Model([inp_1, inp_2], x)

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 6)    0           ['input_6[0][0]',                
                                                                  'input_7[0][0]']                
                                                                                                  
Total params: 0
Trainable params: 0
Non-trainable params: 0
________________________________

# Сверточные слои

### Conv2D

Слой **Conv2D** создает ядро свертки, которое свертывается со входом слоя для получения выходного тензора (пример - свертка изображений).

Входной слой должен иметь три размерности (не считая размерности для батча)  

Основные параметры слоя, влияющие на размер выходного тензора:  
- *filters* - количество нейронов в сверточном слое  
- *kernel_size* - размеры ядра свертки  
- *padding* - позволяет сохранить размерность массива по высоте и ширине не зависимо от размера ядра свертки 
- *strides* - шаги свертки по высоте и ширине  
- *dilation_rate* - параметр для задания разреженной свертки    
  
  
*Формула расчета выходной размерности:*  
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

In [11]:
inp = Input(shape=(128, 128, 3))
x = Conv2D(filters=32, kernel_size=(3,3))(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [12]:
inp = Input(shape=(128, 128, 3))
x = Conv2D(filters=32, kernel_size=(3,3), padding='same')(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


Добавим параметр strides.

In [13]:
inp = Input(shape=(128, 128, 3))
x = Conv2D(filters=32, kernel_size=(3,3), padding='same', strides=(2,2))(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


Добавим параметр dilation_rate.

In [14]:
inp = Input(shape=(128, 128, 3))
x = Conv2D(filters=32, kernel_size=(3,3), padding='valid', dilation_rate=(2,2))(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 32)      896       
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


### MaxPool2D

Слой **MaxPool2D** уменьшает размер входных данных по высоте и ширине, принимая максимальное значение в окне для каждого канала. Размер окна определяется параметром *pool_size*.

Также есть параметр *strides*, который работает так же как и в свертке, но по умолчанию равен размеру окна (pool_size).  


In [15]:
inp = Input(shape=(128, 128, 3))
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(inp)
x = MaxPool2D(pool_size=2)(x)
 
model = Model(inp, x)

model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


Если размеры высоты/ширины не делятся нацело на размер окна, то деление происходит с округлением вниз.

In [16]:
inp = Input(shape=(127, 127, 3))
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(inp)
x = MaxPool2D(pool_size=2)(x)
 
model = Model(inp, x)

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 127, 127, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 127, 127, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


Однако, это можно изменить, если сделать заполнение через *Padding*. 

In [17]:
inp = Input(shape=(127, 127, 3))
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(inp)
x = MaxPool2D(pool_size=2, padding='same')(x)
 
model = Model(inp, x)

model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 127, 127, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 127, 127, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


### UpSampling2D

Слой **UpSampling2D** увеличивает размер тензора, путем повторения данных.

In [18]:
inp = Input(shape=(128, 128, 3))
x = UpSampling2D(size=(2, 2))(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 256, 256, 3)      0         
 )                                                               
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


Пример работы слоя.

In [19]:
inp = Input(shape=(2, 2, 1))
x = UpSampling2D(size=(2, 2))(inp)
 
model = Model(inp, x)

In [20]:
# Создаем данные
data = np.array([[4, 2],
			     [3, 6]])
data

array([[4, 2],
       [3, 6]])

In [21]:
# Модель принимает на вход данные размерности 4D, поэтому приводим к нужному размеру
data = data.reshape((1, 2, 2, 1))

In [22]:
# Подаем данные в модель и возвращаем делаем обратный reshape
pred = model.predict(data)
pred.reshape((4, 4))

1/1 [==============================] - 0s 288ms/step


array([[4., 4., 2., 2.],
       [4., 4., 2., 2.],
       [3., 3., 6., 6.],
       [3., 3., 6., 6.]], dtype=float32)

### Conv2DTranspose

Слой **Conv2DTranspose** также увеличивает размер тензора, но с использованием сверток. Его еще называют Deconvolution-слой.

Он увеличивает размер изображения по высоте и ширине аналогично слою UpSampling2D (только заполняя нулями), а затем применяет свертку к полученному массиву.

Параметр *strides* здесь отвечает за то насколько сильно расширяется исходный массив.

In [23]:
inp = Input(shape=(128, 128, 3))
x = Conv2DTranspose(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same')(inp)
 
model = Model(inp, x)

model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 256, 256, 32)     896       
 nspose)                                                         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


Пример работы слоя.

In [45]:
inp = Input(shape=(2, 2, 1))
x = Conv2DTranspose(filters=1, kernel_size=(1, 1), strides=(3, 3), padding='same')(inp)
 
model = Model(inp, x)

In [46]:
# Создаем данные
data = np.array([[4, 2],
			     [3, 6]])
print(data)

# Модель принимает на вход данные размерности 4D, поэтому приводим к нужному размеру
data = data.reshape((1, 2, 2, 1))

[[4 2]
 [3 6]]


In [52]:
# Для демонстрации установим в модель веса, который никак не влияют на выходные данные
weights = [np.array([[[[1]]]]), np.array([0])]
model.set_weights(weights)


In [27]:
# Подаем данные в модель и возвращаем делаем обратный reshape
pred = model.predict(data)
pred.reshape((6, 6))

1/1 [==============================] - 0s 57ms/step


array([[4., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 6., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

# Слои для работы с последовательностями

### Embedding

Слой **Embedding** создает векторное представление каждого элемента последовательности.

- *input_dim* - размер словаря
- *output_dim* - размерность векторного пространства

In [28]:
inp = Input(shape=(10))
x = Embedding(input_dim=1000, output_dim=5)(inp)
 
model = Model(inp, x)
 
model.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 5)             5000      
                                                                 
Total params: 5,000
Trainable params: 5,000
Non-trainable params: 0
_________________________________________________________________


Пример использования.

In [29]:
data = np.arange(10)
pred = model.predict(data)
print(pred)

1/1 [==============================] - 0s 42ms/step
[[ 0.00193129 -0.00854864 -0.03672947  0.04313781  0.0271039 ]
 [ 0.04726781  0.03637854  0.02126632  0.02061016 -0.01318913]
 [-0.03973256  0.04299212  0.0466626  -0.04153728 -0.01947212]
 [ 0.01582838 -0.01938022 -0.00184568  0.0451187  -0.03858887]
 [ 0.01100177 -0.03717066 -0.04971293 -0.00764813  0.02925721]
 [ 0.00985394 -0.0401486   0.02489301  0.02698697 -0.00691744]
 [ 0.01207183  0.02267965 -0.00673886 -0.00702526 -0.00835854]
 [ 0.01037439 -0.04707965  0.03109381  0.03826543 -0.04105661]
 [-0.02557571 -0.00451971 -0.01754279  0.0238851   0.00831141]
 [-0.0261048  -0.03440655  0.02476935 -0.01002254 -0.04727645]]


### Conv1D

Слой **Conv1D** аналогичен слою Conv2D, но работает с данными меньшей размерности.

In [30]:
inp = Input(shape=(100))
x = Embedding(input_dim=1000, output_dim=20)(inp)
x = Conv1D(filters=32, kernel_size=(5), padding='same')(x)
 
model = Model(inp, x)
 
model.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 100, 20)           20000     
                                                                 
 conv1d (Conv1D)             (None, 100, 32)           3232      
                                                                 
Total params: 23,232
Trainable params: 23,232
Non-trainable params: 0
_________________________________________________________________


### LSTM

<figure>
<center>
<img src='https://hsto.org/web/67b/04f/73b/67b04f73b4c34ba38edfa207e09de07c.png' />
<figcaption>LSTM</figcaption></center>
</figure>

In [31]:
inp = Input(shape=(100))
x = Embedding(input_dim=1000, output_dim=20)(inp)
x = LSTM(units=64)(x)
 
model = Model(inp, x)
 
model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 100)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 100, 20)           20000     
                                                                 
 lstm (LSTM)                 (None, 64)                21760     
                                                                 
Total params: 41,760
Trainable params: 41,760
Non-trainable params: 0
_________________________________________________________________


*return_sequences* - отвечает за то возвращается ли только последний вывод последовательности или вся последовательность.

In [32]:
inp = Input(shape=(4, 1))
x = LSTM(units=1, return_sequences=True)(inp)
model = Model(inp, x)

# Входные данные
data = np.array([0.1, 0.2, 0.3, 0.4]).reshape((1,4,1))

model.predict(data)

1/1 [==============================] - 0s 484ms/step


array([[[-0.00969294],
        [-0.0264568 ],
        [-0.04812365],
        [-0.0728191 ]]], dtype=float32)

In [33]:
inp = Input(shape=(10))
x = Embedding(input_dim=1000, output_dim=20)(inp)
x = LSTM(units=64, return_sequences=True)(x)

model = Model(inp, x)

model.summary()

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 10)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 10, 20)            20000     
                                                                 
 lstm_2 (LSTM)               (None, 10, 64)            21760     
                                                                 
Total params: 41,760
Trainable params: 41,760
Non-trainable params: 0
_________________________________________________________________


In [34]:
data = np.arange(10).reshape(1,10)
model.predict(data).shape

1/1 [==============================] - 1s 610ms/step


(1, 10, 64)

*return_state* - в дополнение к выходным данным возвращает состояние ячейки.  

В итоге возвращается три массива:  
1. Скрытое состояние LSTM для последнего временного шага.
2. Скрытое состояние LSTM для последнего временного шага (еще раз).
3. Состояние ячейки LSTM для последнего временного шага.

Скрытое состояние и состояние ячейки можно использовать для инициализации состояний другого слоя LSTM с тем же количеством ячеек.

In [35]:
inp = Input(shape=(4, 1))
lstm1, state_h, state_c = LSTM(units=1, return_state=True)(inp)
model = Model(inputs=inp, outputs=[lstm1, state_h, state_c])

# Входыне данные
data = np.array([0.1, 0.2, 0.3, 0.4]).reshape((1,4,1))

model.predict(data)

1/1 [==============================] - 1s 727ms/step


[array([[-0.09313004]], dtype=float32),
 array([[-0.09313004]], dtype=float32),
 array([[-0.20073888]], dtype=float32)]

In [36]:
inp = Input(shape=(10))
x = Embedding(input_dim=1000, output_dim=20)(inp)
x = LSTM(units=64, return_state=True)(x)
 
model = Model(inp, x)

model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 10)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 10, 20)            20000     
                                                                 
 lstm_4 (LSTM)               [(None, 64),              21760     
                              (None, 64),                        
                              (None, 64)]                        
                                                                 
Total params: 41,760
Trainable params: 41,760
Non-trainable params: 0
_________________________________________________________________


In [37]:
data = np.arange(10).reshape(1,10)
pred = model.predict(data)
print(len(pred))
print(pred[0].shape, pred[1].shape, pred[2].shape)

1/1 [==============================] - 0s 466ms/step
3
(1, 64) (1, 64) (1, 64)


In [38]:
inp = Input(shape=(4, 1))
lstm1, state_h, state_c = LSTM(units=1, return_state=True)(inp)
model = Model(inputs=inp, outputs=[lstm1, state_h, state_c])

# Входыне данные
data = np.array([0.1, 0.2, 0.3, 0.4]).reshape((1,4,1))

model.predict(data)

1/1 [==============================] - 0s 370ms/step


[array([[-0.12626255]], dtype=float32),
 array([[-0.12626255]], dtype=float32),
 array([[-0.2661676]], dtype=float32)]

return_state + return_sequences

In [39]:
inp = Input(shape=(4, 1))
lstm1, state_h, state_c = LSTM(units=1, return_sequences=True, return_state=True)(inp)
model = Model(inputs=inp, outputs=[lstm1, state_h, state_c])

# Входыне данные
data = np.array([0.1, 0.2, 0.3, 0.4]).reshape((1,4,1))

model.predict(data)

1/1 [==============================] - 1s 539ms/step


[array([[[-0.01553788],
         [-0.03938013],
         [-0.06631017],
         [-0.09259953]]], dtype=float32),
 array([[-0.09259953]], dtype=float32),
 array([[-0.23518857]], dtype=float32)]

In [40]:
inp = Input(shape=(10))
x = Embedding(input_dim=1000, output_dim=20)(inp)
x = LSTM(64, return_state=True, return_sequences=True, activation='softmax') (x)
 
model = Model(inp, x)

model.summary()

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 10)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 10, 20)            20000     
                                                                 
 lstm_7 (LSTM)               [(None, 10, 64),          21760     
                              (None, 64),                        
                              (None, 64)]                        
                                                                 
Total params: 41,760
Trainable params: 41,760
Non-trainable params: 0
_________________________________________________________________


In [41]:
data = np.arange(10).reshape(1,10)
pred = model.predict(data)
print(len(pred))
print(pred[0].shape, pred[1].shape, pred[2].shape)

1/1 [==============================] - 0s 250ms/step
3
(1, 10, 64) (1, 64) (1, 64)
